In [23]:
import alpaca_trade_api as tradeapi
from helper_functions import *
from cam_paper_keys import *
import pandas as pd
import numpy as np
import nltk
nltk.download('vader_lexicon', quiet=True)
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import bs4 as bs
import requests
from datetime import timedelta

In [2]:
api = connect_paper_api(paper_key_id, paper_secret_key)
symbols = save_sp500_tickers()
sia = SentimentIntensityAnalyzer()

In [5]:
resp = requests.get(news.url)

In [ ]:
str(pd.Timestamp.today() - )[0:10]

In [25]:
todays_date = str(pd.Timestamp.today())[0:10]
yesterdays_date = str(pd.Timestamp.today() - timedelta(1))[0:10]
for sym in symbols[0:50]:
    for news in api.polygon.news(sym)[0:5]:
        if (news.timestamp[0:10] == todays_date) or (news.timestamp[0:10] == yesterdays_date):
            resp = requests.get(news.url)
            try:
                soup = bs.BeautifulSoup(resp.text, 'html.parser')
                summary = soup.find('div', {'class': 'a-sum'})
                string = []
                [string.append(blurb.text) for blurb in summary.findAll('p')]
                string = ' '.join(string)
                print('Symbol: ', sym, '\nSummary: ', string, '\nScore: ', sia.polarity_scores(string)['compound'])
            except:
                pass
        else:
            pass 

Symbol:  ABBV 
Summary:  Humira had to be discounted by over 80% overseas and a similar plight could occur in the US sooner than later. The stock has upside potential but considerable downside risk. Much depends on ramping up of drugs in the pipeline and FDA approvals for new drugs. From a valuation perspective, the stock has 24% upside to 17% downside and in our opinion, this isn't very compelling at the moment. Speculative investors may want to take a shot but conservative investors should sit this one out. 
Score:  -0.7798
Symbol:  ATVI 
Summary:  There is a strong lineup set for full-year 2019; expected to expand rapidly moving into 2020. An active emphasis is now placed on the expansion of technical development teams, capacity up by 20% leading to higher quality releases. Legacy management issues are being addressed competently and the return of previous CFO provides positive market sentiment. This is an attractive price point to buy into a great company at a fair price. 
Score:  